In [1]:
import pandas as pd
import numpy as np  # noqa
import optuna
import time
import logging  # noqa
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error
from lightgbm import LGBMRegressor, early_stopping, log_evaluation

# Enable logging
optuna.logging.set_verbosity(optuna.logging.INFO)


class TimingCallback:
    def __init__(self):
        self.start_time = None

    def __call__(self, study, trial):
        if self.start_time is None:
            self.start_time = time.time()
        else:
            elapsed_time = time.time() - self.start_time
            print(f"Trial {trial.number} finished in {elapsed_time:.2f} seconds.")
            self.start_time = time.time()

/home/lsf/anaconda3/envs/Pein_310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data_dir = "/data3/lsf/Pein/Power-Prediction/data/"
train_data_selected = pd.read_csv(data_dir + "train_farm_92.csv")
test_data_selected = pd.read_csv(data_dir + "test_farm_92.csv")

# Define features and target
features = [
    col
    for col in train_data_selected.columns
    if col not in ["time", "lead_hour", "power"]
]
X_train = train_data_selected[features]
y_train = train_data_selected["power"]
X_test = test_data_selected[features]
y_test = test_data_selected["power"]

### Adding time features

In [3]:
# Convert 'time' column to datetime
train_data_selected["time"] = pd.to_datetime(train_data_selected["time"])
test_data_selected["time"] = pd.to_datetime(test_data_selected["time"])

In [4]:
# Function to add time features
def add_time_features(df):
    df["time"] = pd.to_datetime(df["time"])

    # Existing time features
    df["hour"] = df["time"].dt.hour
    df["quarter_hour"] = df["time"].dt.minute // 15

    # New time features
    df["day"] = df["time"].dt.day
    df["day_in_week"] = df["time"].dt.weekday

    # Sine and cosine transformations
    df["hour_sin"] = np.sin(2 * np.pi * df["hour"] / 24)
    df["hour_cos"] = np.cos(2 * np.pi * df["hour"] / 24)

    df["quarter_hour_sin"] = np.sin(2 * np.pi * df["quarter_hour"] / 4)
    df["quarter_hour_cos"] = np.cos(2 * np.pi * df["quarter_hour"] / 4)

    df["day_sin"] = np.sin(2 * np.pi * df["day"] / 31)
    df["day_cos"] = np.cos(2 * np.pi * df["day"] / 31)

    df["day_in_week_sin"] = np.sin(2 * np.pi * df["day_in_week"] / 7)
    df["day_in_week_cos"] = np.cos(2 * np.pi * df["day_in_week"] / 7)

    return df

In [5]:
# Add time features to both train and test data
train_data_selected = add_time_features(train_data_selected)
test_data_selected = add_time_features(test_data_selected)

In [6]:
# Define features including the new time features
time_features = [
    "hour",
    "quarter_hour",
    "day",
    "day_in_week",
    "hour_sin",
    "hour_cos",
    "quarter_hour_sin",
    "quarter_hour_cos",
    "day_sin",
    "day_cos",
    "day_in_week_sin",
    "day_in_week_cos",
]

In [7]:
all_features = features + time_features

X_train = train_data_selected[all_features]
X_test = test_data_selected[all_features]

In [8]:
def objective(
    trial,
    X_train,
    y_train,
    X_test=None,
    y_test=None,
    use_test_for_validation_flag=False,
):
    params = {
        "objective": "regression",
        "metric": "rmse",
        "boosting_type": "gbdt",
        "num_leaves": trial.suggest_int("num_leaves", 10, 80),
        "max_depth": trial.suggest_int("max_depth", 10, 50),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.2),
        "n_estimators": trial.suggest_int("n_estimators", 500, 1500),
        "min_child_samples": trial.suggest_int("min_child_samples", 10, 50),
        "subsample": trial.suggest_float("subsample", 0.3, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "num_threads": 16,
        "seed": 42,
    }

    model = LGBMRegressor(**params)

    if use_test_for_validation_flag and X_test is not None and y_test is not None:
        X_val_split, y_val_split = X_test, y_test
        model.fit(
            X_train,
            y_train,
            eval_set=[(X_val_split, y_val_split)],
            eval_metric="rmse",
            callbacks=[early_stopping(stopping_rounds=15), log_evaluation(period=500)],
        )
        preds = model.predict(X_val_split)
    else:
        X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(
            X_train, y_train, test_size=0.2, random_state=42
        )
        model.fit(
            X_train_split,
            y_train_split,
            eval_set=[(X_val_split, y_val_split)],
            eval_metric="rmse",
            callbacks=[early_stopping(stopping_rounds=15), log_evaluation(period=500)],
        )
        preds = model.predict(X_val_split)

    # Calculate RMSE
    rmse = root_mean_squared_error(y_val_split, preds)
    return rmse

In [9]:
# Create the study and optimize
use_test_for_validation_flag = False
study = optuna.create_study(direction="minimize")
study.optimize(
    lambda trial: objective(
        trial,
        X_train,
        y_train,
        X_test,
        y_test,
        use_test_for_validation_flag=use_test_for_validation_flag,
    ),
    n_trials=50,
    callbacks=[TimingCallback()],
)

# Get the best parameters
best_params = study.best_params
print("Best parameters found: ", best_params)

[I 2024-08-02 15:26:13,776] A new study created in memory with name: no-name-3638510d-4c6e-4ca8-aa46-0683768eebbd


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002369 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 22879
[LightGBM] [Info] Number of data points in the train set: 11673, number of used features: 101
[LightGBM] [Info] Start training from score 100888.355472
Training until validation scores don't improve for 15 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best g

[I 2024-08-02 15:26:17,490] Trial 0 finished with value: 12569.131255407501 and parameters: {'num_leaves': 70, 'max_depth': 10, 'learning_rate': 0.11089815188227202, 'n_estimators': 1050, 'min_child_samples': 19, 'subsample': 0.8658201320230681, 'colsample_bytree': 0.5497136054577744}. Best is trial 0 with value: 12569.131255407501.


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002803 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 22879
[LightGBM] [Info] Number of data points in the train set: 11673, number of used features: 101
[LightGBM] [Info] Start training from score 100888.355472
Training until validation scores don't improve for 15 rounds
[500]	valid_0's rmse: 12895.6
[1000]	valid_0's rmse: 12526.6


[I 2024-08-02 15:26:23,255] Trial 1 finished with value: 12467.48954665738 and parameters: {'num_leaves': 77, 'max_depth': 40, 'learning_rate': 0.09744597713054322, 'n_estimators': 1401, 'min_child_samples': 27, 'subsample': 0.9853734683934467, 'colsample_bytree': 0.6710221845532004}. Best is trial 1 with value: 12467.48954665738.


...

Did not meet early stopping. Best iteration is:
[1336]	valid_0's rmse: 12542.7
Trial 47 finished in 5.09 seconds.
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002561 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 22879
[LightGBM] [Info] Number of data points in the train set: 11673, number of used features: 101
[LightGBM] [Info] Start training from score 100888.355472
Training until validation scores don't improve for 15 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[500]	valid_0's rmse: 14156
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -in

[I 2024-08-02 15:29:17,234] Trial 48 finished with value: 12653.046629387734 and parameters: {'num_leaves': 76, 'max_depth': 26, 'learning_rate': 0.04172143771147141, 'n_estimators': 1409, 'min_child_samples': 35, 'subsample': 0.41229188722947296, 'colsample_bytree': 0.6645197754639679}. Best is trial 44 with value: 12263.373537879055.


Trial 48 finished in 5.71 seconds.
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002467 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 22879
[LightGBM] [Info] Number of data points in the train set: 11673, number of used features: 101
[LightGBM] [Info] Start training from score 100888.355472
Training until validation scores don't improve for 15 rounds
[500]	valid_0's rmse: 12895.4
[1000]	valid_0's rmse: 12573.9


[I 2024-08-02 15:29:22,708] Trial 49 finished with value: 12525.717246476153 and parameters: {'num_leaves': 72, 'max_depth': 26, 'learning_rate': 0.11805269103023541, 'n_estimators': 1463, 'min_child_samples': 25, 'subsample': 0.5562182418998072, 'colsample_bytree': 0.7198213659234672}. Best is trial 44 with value: 12263.373537879055.


Early stopping, best iteration is:
[1419]	valid_0's rmse: 12525.7
Trial 49 finished in 5.47 seconds.
Best parameters found:  {'num_leaves': 79, 'max_depth': 26, 'learning_rate': 0.07455914521078384, 'n_estimators': 1446, 'min_child_samples': 22, 'subsample': 0.43544652153306895, 'colsample_bytree': 0.631599023200065}


In [10]:
# best_params = {
#     "num_leaves": 70,
#     "max_depth": 36,
#     "learning_rate": 0.09304350950671668,
#     "n_estimators": 1158,
#     "min_child_samples": 18,
#     "subsample": 0.579731306036922,
#     "colsample_bytree": 0.8511910376678277,
# }

In [11]:
print("Best parameters found: ", best_params)
best_params["num_threads"] = 16

# Train the model with the best parameters
best_model = LGBMRegressor(**best_params)
best_model.fit(X_train, y_train)


# Extract feature importances
feature_importances = best_model.feature_importances_

# Create a DataFrame to hold the features and their importances
importance_df = pd.DataFrame(
    {"Feature": X_train.columns, "Importance": feature_importances}
)

# Sort the DataFrame by importance
importance_df = importance_df.sort_values(by="Importance", ascending=False)

Best parameters found:  {'num_leaves': 79, 'max_depth': 26, 'learning_rate': 0.07455914521078384, 'n_estimators': 1446, 'min_child_samples': 22, 'subsample': 0.43544652153306895, 'colsample_bytree': 0.631599023200065}
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002785 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 22879
[LightGBM] [Info] Number of data points in the train set: 14592, number of used features: 101
[LightGBM] [Info] Start training from score 101382.392957


In [12]:
# Predict on the test set
y_pred = best_model.predict(X_test)

# Calculate RMSE
rmse = root_mean_squared_error(y_test, y_pred)
print(f"RMSE on the test set: {rmse:.4f}")

RMSE on the test set: 65581.0135


In [13]:
# save important_features to csv file
if use_test_for_validation_flag:
    importance_df.to_csv("farm_important_features_use_test_for_validation.csv", index=False)
else:
    importance_df.to_csv("farm_important_features.csv", index=False)

In [14]:
# Set the threshold for feature importance
importance_threshold = 200

# Select features with importance greater than or equal to the threshold
selected_features = importance_df[importance_df["Importance"] >= importance_threshold][
    "Feature"
].tolist()

# Separate selected features into original input features and time features
selected_input_features = [
    feature for feature in selected_features if feature not in time_features
]
selected_time_features = [
    feature for feature in selected_features if feature in time_features
]

In [15]:
# Combine the filtered features with the power column and time features
train_data_final = pd.concat(
    [
        train_data_selected["time"],
        X_train[selected_input_features],
        train_data_selected["power"],
        train_data_selected[time_features],
    ],
    axis=1,
)
test_data_final = pd.concat(
    [
        test_data_selected["time"],
        X_test[selected_input_features],
        test_data_selected["power"],
        test_data_selected[time_features],
    ],
    axis=1,
)

# Ensure the final order of columns
final_columns = ["time"] + selected_input_features + ["power"] + time_features
train_data_final = train_data_final[final_columns]
test_data_final = test_data_final[final_columns]

# Display the new shapes of train_data_final and test_data_final
print("Shape of train_data_final:", train_data_final.shape)
print("Shape of test_data_final:", test_data_final.shape)
print("Selected input features:", selected_input_features)
print(f"Train columns : {train_data_final.columns}")

Shape of train_data_final: (14592, 97)
Shape of test_data_final: (2880, 97)
Selected input features: ['lgws', 'mgws', 'altitude', 'azimuth', 'trpp', 'blh', 'mudlp', 'lcc', 'deg0l', 'mcc', 'kx', 'totalx', 'tcc', 'viwvn', 'tcsw', 'flsr', 'fg10', 'capes', 'ilspf', 'ishf', 'vimd', 'viwve', 'bld', 'degm10l', 'u100', 'p3020', 'gwd', 'ws200', 'ewss', 'mld', 'cape', 'ws10', 'ws100', 'lspf', 'hcc', 'litoti', 'i10fg', 'u200', 'u10', 'u10n', 'mlcape100', 'hwbt0', 'sund', 'msl', 'hwbt1', 'dsrp', 'nsss', 'fdir', 'mucape', 'mlcape50', 'str', 'sshf', 'v200', 'ttrc', 'tcw', 'cdir', 'v100', 'v10', 'ttr', 'slhf', 'v10n', 'par', 'strc', 'd2m', 'tsr', 't2m', 'parcs', 'ssrd', 'uvb', 'sp', 'tisr', 'ssrdc', 'ssrc', 'strd', 'ssr', 'tsrc', 'tcwv', 'mn2t', 'mx2t', 'lblt', 'sst', 'skt', 'strdc']
Train columns : Index(['time', 'lgws', 'mgws', 'altitude', 'azimuth', 'trpp', 'blh', 'mudlp',
       'lcc', 'deg0l', 'mcc', 'kx', 'totalx', 'tcc', 'viwvn', 'tcsw', 'flsr',
       'fg10', 'capes', 'ilspf', 'ishf', 'vimd',

In [16]:
# Save the final datasets to CSV
train_data_final.to_csv(data_dir + "train_farm_97_withTime.csv", index=False)
test_data_final.to_csv(data_dir + "test_farm_97_withTime.csv", index=False)
print("Final datasets saved to CSV.")

Final datasets saved to CSV.
